# Cuaderno 01: Generación de Datasets y Embeddings

### Objetivo 🎯
Este cuaderno es el punto de partida fundamental de nuestro proyecto. Su objetivo es construir de manera sistemática y robusta todos los **datasets paralelos** (auditivo y visual) y las **representaciones numéricas (embeddings)** que servirán de base para el entrenamiento de los modelos.

### Flujo de Trabajo Consolidado ⚙️

1.  **Configuración e Instalación**: Definiremos los parámetros globales, incluyendo el **número de palabras deseadas** para el experimento y el **modelo `wav2vec2-large`** para asegurar embeddings de dimensión 1024.
2.  **Preprocesamiento - Generación de Diccionarios (Robusto)**: Una nueva sección automática que:
    - Descarga corpus de palabras para español e inglés.
    - Filtra un gran número de palabras candidatas que cumplen nuestros criterios (solo letras válidas).
    - **Selecciona aleatoriamente** el número deseado de palabras para garantizar la diversidad léxica.
    - Ordena alfabéticamente la lista final y la guarda, creando diccionarios fiables.
3.  **Generación del Dataset Visual (EMNIST)**: Descarga y extrae imágenes de letras (`.png`) del dataset EMNIST, con un **límite de 10 ejemplos por grafema** para mantener el dataset balanceado.
4.  **Generación de Audio (gTTS)**: Crea archivos `.wav` para cada fonema y para cada palabra de nuestros nuevos y diversos diccionarios.
5.  **Extracción de Embeddings (Wav2Vec2)**: Procesa todos los archivos `.wav` para crear su "imagen auditiva", asegurando que se extraiga el **`last_hidden_state`** para obtener los embeddings de dimensión 1024.
6.  **Verificación Final**: Se añade un paso final que **verifica en el disco duro** que todos los activos necesarios para las palabras del diccionario se hayan creado con éxito, garantizando que no habrá errores de archivos faltantes en los cuadernos posteriores.


In [23]:
# ===================================================================
# Celda 2: Configuración e Instalación
# ===================================================================
%pip install torch transformers gtts pandas numpy scikit-learn nltk unidecode torchvision --quiet

import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from torchvision.datasets import EMNIST
from torchvision.transforms import ToPILImage
from gtts import gTTS
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import os
import re
import nltk
from unidecode import unidecode
import shutil
import random
from PIL import Image
import time

Note: you may need to restart the kernel to use updated packages.


In [29]:
# ===================================================================
# Celda 3: Parámetros Globales
# ===================================================================
# --- Parámetros del Experimento ---
# Define con cuántas palabras VÁLIDAS Y VERIFICADAS quieres que termine el proceso.
NUM_FINAL_WORDS = 500
# Buscaremos un número mayor de candidatos para tener margen por si falla la generación de algún archivo.
NUM_CANDIDATE_WORDS = int(NUM_FINAL_WORDS * 2) 
MAX_EXAMPLES_PER_GRAPHEME = 10
LANGUAGES = ['es', 'en']

# Fonemas/Grafemas a generar (25 letras sin 'ñ' para compatibilidad)
PHONEMES = {
    'es': sorted(list('abcdefghijklmnopqrstuvwxyz')),
    'en': sorted(list('abcdefghijklmnopqrstuvwxyz'))
}

# --- Rutas del Proyecto ---
project_root = Path.cwd().parent
raw_data_dir = project_root / "data/01_raw"
dictionaries_dir = raw_data_dir / "dictionaries"
output_audio_dir = project_root / "data/02_processed/audio"
output_embedding_dir = project_root / "data/02_processed/wav2vec2_embeddings"
output_visual_dir = project_root / "data/02_processed/visual"
output_word_audio_dir = project_root / "data/02_processed/word_audio"
output_word_embedding_dir = project_root / "data/02_processed/word_embeddings"

# --- Creación de Directorios ---
for path in [dictionaries_dir, output_audio_dir, output_embedding_dir, output_visual_dir, output_word_audio_dir, output_word_embedding_dir]:
    path.mkdir(parents=True, exist_ok=True)

# --- Configuración del Modelo de Embeddings ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Se usa el modelo 'large' para asegurar embeddings de dimensión 1024
MODEL_ID = "facebook/wav2vec2-large-960h-lv60-self"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(DEVICE)
EMBED_DIM = model.config.hidden_size
print(f"Modelo Wav2Vec2 cargado: '{MODEL_ID}'")
print(f"Dimensión de embedding confirmada: {EMBED_DIM}")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo Wav2Vec2 cargado: 'facebook/wav2vec2-large-960h-lv60-self'
Dimensión de embedding confirmada: 1024


In [25]:
# ===================================================================
# Celda 4: Preprocesamiento - Generación de Lista de PALABRAS CANDIDATAS
# ===================================================================
def generate_candidate_lists(num_candidates, languages):
    print("Descargando corpus de NLTK (puede tardar la primera vez)...")
    nltk.download('words', quiet=True); nltk.download('cess_esp', quiet=True)
    print("Corpus descargados.")
    valid_chars = set('abcdefghijklmnopqrstuvwxyz')
    word_sources = {'en': set(nltk.corpus.words.words()), 'es': set(nltk.corpus.cess_esp.words())}
    candidate_words = {}
    
    for lang in languages:
        print(f"\n--- Procesando diccionario de candidatos para: {lang.upper()} ---")
        source_words = word_sources.get(lang, set())
        print(f"  Corpus original cargado con {len(source_words)} palabras.")
        valid_words = {unidecode(w.lower()) for w in source_words if len(unidecode(w.lower())) > 2 and set(unidecode(w.lower())).issubset(valid_chars)}
        print(f"  Se encontraron {len(valid_words)} palabras válidas después del filtro.")
        if len(valid_words) >= num_candidates:
            # Seleccionar aleatoriamente para asegurar la variedad.
            candidate_words[lang] = random.sample(sorted(list(valid_words)), num_candidates)
            print(f"  ✅ Éxito: Se seleccionaron {len(candidate_words[lang])} palabras candidatas.")
        else:
            print(f"  ❌ ADVERTENCIA: No se encontraron suficientes palabras candidatas (Se necesitan {num_candidates}).")
            candidate_words[lang] = []
    return candidate_words

CANDIDATE_WORDS = generate_candidate_lists(NUM_CANDIDATE_WORDS, LANGUAGES)

Descargando corpus de NLTK (puede tardar la primera vez)...
Corpus descargados.

--- Procesando diccionario de candidatos para: ES ---
  Corpus original cargado con 25464 palabras.
  Se encontraron 19644 palabras válidas después del filtro.
  ✅ Éxito: Se seleccionaron 1000 palabras candidatas.

--- Procesando diccionario de candidatos para: EN ---
  Corpus original cargado con 235892 palabras.
  Se encontraron 234210 palabras válidas después del filtro.
  ✅ Éxito: Se seleccionaron 1000 palabras candidatas.


In [26]:
# ===================================================================
# Celda 5: Generación del Dataset Visual (EMNIST)
# ===================================================================
def generate_visual_dataset(output_dir, raw_dir, max_examples=10):
    print("\n--- Generando Dataset Visual (EMNIST) ---")
    emnist_dataset = EMNIST(root=raw_dir, split='letters', download=True, train=True)
    label_map = {i: chr(ord('a') + i - 1) for i in range(1, 27)}
    if output_dir.exists(): shutil.rmtree(output_dir)
    output_dir.mkdir(parents=True)
    counts = {chr(ord('a') + i): 0 for i in range(26)}
    for image, label_idx in tqdm(emnist_dataset, desc="Procesando imágenes de EMNIST"):
        label_char = label_map.get(label_idx)
        if label_char and counts[label_char] < max_examples:
            char_dir = output_dir / label_char
            char_dir.mkdir(exist_ok=True)
            pil_image = image.rotate(-90, expand=True).transpose(Image.FLIP_LEFT_RIGHT)
            counts[label_char] += 1
            image_path = char_dir / f"{label_char}_{counts[label_char]}.png"
            pil_image.save(image_path)
    print("✅ Dataset Visual EMNIST generado con éxito.")

generate_visual_dataset(output_visual_dir, raw_data_dir, max_examples=MAX_EXAMPLES_PER_GRAPHEME)


--- Generando Dataset Visual (EMNIST) ---


Procesando imágenes de EMNIST:   0%|          | 0/124800 [00:00<?, ?it/s]

✅ Dataset Visual EMNIST generado con éxito.


In [30]:
# ===================================================================
# Celda 6: Generación de Audio para Fonemas y Palabras
# ===================================================================
def generate_audio(text_list, output_dir, lang):
    output_dir.mkdir(parents=True, exist_ok=True)
    for text in tqdm(text_list, desc=f"Generando audio para '{lang}' en '{output_dir.name}'"):
        file_path = output_dir / f"{text}.wav"
        time.sleep(5)
        if not file_path.exists():
            try:
                gTTS(text, lang=lang).save(str(file_path))
            except Exception as e:
                print(f"No se pudo generar audio para '{text}': {e}")

for lang in LANGUAGES:
    generate_audio(PHONEMES[lang], output_audio_dir / lang, lang)
    if CANDIDATE_WORDS[lang]:
        generate_audio(CANDIDATE_WORDS[lang], output_word_audio_dir / lang, lang)
        

Generando audio para 'es' en 'es':   0%|          | 0/26 [00:00<?, ?it/s]

Generando audio para 'es' en 'es':   0%|          | 0/1000 [00:00<?, ?it/s]

No se pudo generar audio para 'acumulacion': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'contemplaban': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'dimension': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'arrojados': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'establecimiento': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'conmover': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'vigor': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'intocables': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'relata': 429 (Too Many Requests) from TTS API. Probable cause: Unknown
No se pudo generar audio para 'bell': 429 (Too Many

KeyboardInterrupt: 

In [ ]:
# ===================================================================
# Celda 7: Extracción de Embeddings Auditivos (Wav2Vec2)
# ===================================================================
import librosa
def extract_embeddings(audio_dir, embedding_dir):
    embedding_dir.mkdir(parents=True, exist_ok=True)
    audio_files = list(audio_dir.glob("*.wav"))
    for audio_path in tqdm(audio_files, desc=f"Extrayendo embeddings de '{audio_dir.name}'"):
        text = audio_path.stem
        embedding_path = embedding_dir / f"{text}.npy"
        if not embedding_path.exists():
            try:
                speech_array, sr = librosa.load(str(audio_path), sr=16000)
                inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt", padding=True)
                with torch.no_grad():
                    outputs = model.wav2vec2(inputs.input_values.to(DEVICE))
                    hidden_states = outputs.last_hidden_state
                np.save(embedding_path, hidden_states.cpu().numpy().squeeze())
            except Exception as e:
                print(f"No se pudo extraer embedding para '{text}': {e}")

for lang in LANGUAGES:
    extract_embeddings(output_audio_dir / lang, output_embedding_dir / lang)
    if CANDIDATE_WORDS[lang]:
        extract_embeddings(output_word_audio_dir / lang, output_word_embedding_dir / lang)

Extrayendo embeddings de 'es':   0%|          | 0/26 [00:00<?, ?it/s]

Extrayendo embeddings de 'es':   0%|          | 0/788 [00:00<?, ?it/s]

Extrayendo embeddings de 'en':   0%|          | 0/26 [00:00<?, ?it/s]

Extrayendo embeddings de 'en':   0%|          | 0/799 [00:00<?, ?it/s]

In [33]:
# ===================================================================
# Celda 8: Creación de Diccionarios Basada en Embeddings Existentes (Corregida)
# ===================================================================
from pathlib import Path # Importar Path para manejar las rutas

print("\n--- Creación de Diccionarios a partir de Embeddings Verificados ---")

# --- ¡NUEVO! Definición de rutas directas ---
# Asumimos que el notebook se ejecuta desde una carpeta como 'notebooks/03_auditory_pathway.ipynb'
# y los datos están en 'data/', por lo que subimos un nivel.
project_root = Path.cwd().parent 
phoneme_embedding_dir_base = project_root / "data/02_processed/wav2vec2_embeddings"
word_embedding_dir_base = project_root / "data/02_processed/word_embeddings"

for lang in LANGUAGES:
    print(f"\n--- Procesando idioma: {lang.upper()} ---")
    
    # Directorios específicos para el idioma actual
    phoneme_dir = phoneme_embedding_dir_base / lang
    word_dir = word_embedding_dir_base / lang
    
    # Verificar si los directorios existen para evitar errores
    if not word_dir.exists() or not phoneme_dir.exists():
        print(f" ❌ ERROR: No se encontraron los directorios de embeddings para '{lang}'.")
        print(f"   - Buscando en: {word_dir}")
        print(f"   - Buscando en: {phoneme_dir}")
        print("   Saltando este idioma...")
        continue

    # 1. Obtener todas las palabras candidatas directamente de los archivos .npy existentes.
    word_candidates = [p.stem for p in word_dir.glob("*.npy")]
    
    verified_words = []
    
    # 2. Filtrar la lista: verificar que para cada palabra, todos sus fonemas (letras) también existen.
    print(f"Encontrados {len(word_candidates)} embeddings de palabras. Verificando fonemas correspondientes...")
    for word in tqdm(word_candidates, desc=f"Verificando activos ({lang})"):
        all_phonemes_exist = all((phoneme_dir / f"{phoneme}.npy").exists() for phoneme in word)
        
        if all_phonemes_exist:
            verified_words.append(word)

    # 3. Generar el diccionario con TODAS las palabras que pasaron el filtro.
    if verified_words:
        final_list = sorted(verified_words)
        
        # Asegurarse de que el directorio de diccionarios exista
        dictionaries_dir.mkdir(parents=True, exist_ok=True)
        dict_path = dictionaries_dir / f"{lang}_words.txt"
        
        with open(dict_path, 'w', encoding='utf-8') as f:
            for word in final_list:
                f.write(f"{word}\n")
                
        # 4. Informar el resultado final con el recuento total.
        print(f" ✅ Éxito: Se generó el diccionario '{dict_path.name}' con un total de {len(final_list)} palabras verificadas.")
    else:
        print(f" ⚠️ Advertencia: No se encontró ninguna palabra que cumpliera con los requisitos para '{lang}'. No se generó el diccionario.")

print("\n\n✅ Proceso de creación de diccionarios finalizado.")


--- Creación de Diccionarios a partir de Embeddings Verificados ---

--- Procesando idioma: ES ---
Encontrados 1169 embeddings de palabras. Verificando fonemas correspondientes...


Verificando activos (es):   0%|          | 0/1169 [00:00<?, ?it/s]

 ✅ Éxito: Se generó el diccionario 'es_words.txt' con un total de 1169 palabras verificadas.

--- Procesando idioma: EN ---
Encontrados 1197 embeddings de palabras. Verificando fonemas correspondientes...


Verificando activos (en):   0%|          | 0/1197 [00:00<?, ?it/s]

 ✅ Éxito: Se generó el diccionario 'en_words.txt' con un total de 1197 palabras verificadas.


✅ Proceso de creación de diccionarios finalizado.
